In [1]:
# !pip3 install fast_bert

In [2]:
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging
from fast_bert.prediction import BertClassificationPredictor
import torch

logger = logging.getLogger()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False


metrics = [{'name': 'accuracy', 'function': accuracy}]

OUTPUT_DIR = "Model_Artifacts"
DATA_PATH = "Data"
LABEL_PATH = "labels" 

### Dataloading
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='xlnet-base-cased',
                          train_file='Train.csv',
                          val_file='Val.csv',
                          label_file='labels.csv',
                          text_col='questionText',
                          label_col='TherapyPersona',
                          batch_size_per_gpu=4,
                          max_seq_length=124,
                          multi_gpu=multi_gpu,
                          multi_label=False,
                          model_type='xlnet')

### Learner Creation
learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='pytorchdump',
						metrics=metrics,
						device=device,
						logger=logger,
						output_dir=OUTPUT_DIR,
						finetuned_wgts_path=None,
						warmup_steps=100,
						multi_gpu=multi_gpu,
						is_fp16=False,
						multi_label=False,
						logging_steps=5)

### Train and Save
learner.fit(epochs=1,
			lr=6e-6,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine",
			optimizer_type="lamb")

learner.save_model()

### Inference
texts = ['this is a test','I sure love tests']
predictions = learner.predict_batch(texts)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
